In [1]:
from utils import FileDataset, AnomalyDataset, cache
from net import SpatialEncoder
from pathlib import Path
import os
import torch
import torch.nn as nn
import wandb
import tqdm
import gc
import numpy as np
import scipy.io as sio
import pandas as pd
import numpy as np
import torch
from pathlib import Path
import os
import matplotlib.pyplot as plt
import tqdm
import torch.nn as nn
from torch.utils.data import Dataset
from collections import namedtuple
import tqdm
from torchvision.transforms import Resize, Compose, ToPILImage, ToTensor
from PIL import Image
import torch.nn.functional as F
from conv_lstm import ConvLSTM
import neptune
import wandb
from PIL import Image
from matplotlib import cm



In [2]:
wandb.init(project='anomaly', entity='coinitialized')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: damiankucharski (use `wandb login --relogin` to force relogin)


In [3]:
train_path = Path('Avenue Dataset/training_vol')
test_path = Path('Avenue Dataset/testing_vol')

In [4]:
device = torch.device('cuda')
model = SpatialEncoder()
model = model.to(device)

In [5]:
optimizer = torch.optim.Adam(model.parameters())
loss = nn.MSELoss()
epochs = 50

In [6]:
set_train = FileDataset(train_path, test_path, train=True) 
set_test = FileDataset(train_path, test_path, train=False)

train_data_loader = torch.utils.data.DataLoader(set_train, batch_size=1, shuffle=True)
test_data_loader = torch.utils.data.DataLoader(set_test, batch_size=1, shuffle=True)

In [7]:
losses_train = []
losses_test = []

In [8]:
for i in tqdm.tqdm(range(epochs)):

    print('train')
    batch_loss_train = []
    model.train()

    batch_loss_train = []
    for X in tqdm.tqdm(train_data_loader):


        print(type(X))
        print(X.size())

        set_train = AnomalyDataset(X[0], train=True) 
        print(len(set_train))
        train_loader = torch.utils.data.DataLoader(set_train, batch_size=12, shuffle=True)
        for X_batch in train_loader:
            optimizer.zero_grad()
            X_batch = X_batch.to(device)
            predictions = model(X_batch)
            loss_val = loss(predictions, X_batch)
            batch_loss_train.append(loss_val.item())
            loss_val.backward()
            optimizer.step()
            del X_batch
            gc.collect()
        del X
        gc.collect()

    losses_train.append(np.mean(batch_loss_train))

    print('test')
    batch_loss_test = []
    for X in tqdm.tqdm(test_data_loader):
        
        set_test = AnomalyDataset(X[0], train=True) 
        test_loader = torch.utils.data.DataLoader(set_train, batch_size=12, shuffle=True)

        model.eval()
        batch_loss_test = []
        for X_batch in test_loader:
            X_batch = X_batch.to(device)
            predictions = model(X_batch)
            loss_val = loss(predictions, X_batch)
            batch_loss_test.append(loss_val.item())
            del X_batch
            gc.collect()
        del X
        gc.collect()
    
    losses_test.append(np.mean(batch_loss_train))

    output_image = predictions[:1,...]
    output_image = output_image.squeeze()
    one_frame = output_image[0,...]
    one_frame_numpy = one_frame.cpu().detach().numpy()
    im = Image.fromarray(np.uint8(cm.gist_earth(one_frame_numpy)*255))

    wandb.log({"loss_train": losses_train[-1],"loss_test": losses_test[-1], "epoch": i,
            "inputs": wandb.Image(im),
            })


    PATH = f"checkpoints/{i}_model.pt"

    torch.save({
            'epoch': i,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss_train': losses_train[-1],
            'loss_test' : losses_test[-1]
            }, PATH)

    if losses_test[i-5] - losses_test[i-1] < 0.00025:
      break



  0%|          | 0/16 [00:00<?, ?it/s]train
Generating strides for Avenue Dataset\training_vol\vol02.mat
Generating strides for Avenue Dataset\training_vol\vol02.mat
Generating strides for Avenue Dataset\training_vol\vol02.mat
<class 'torch.Tensor'>
torch.Size([1, 2753, 1, 10, 227, 227])
2202
C:\Users\cdami\Anaconda3\lib\site-packages\torch\nn\functional.py:1628: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
C:\Users\cdami\Anaconda3\lib\site-packages\torch\nn\functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")

  6%|▋         | 1/16 [03:03<45:45, 183.05s/it]Generating strides for Avenue Dataset\training_vol\vol01.mat
Generating strides for Avenue Dataset\training_vol\vol01.mat
Generating strides for Avenue Dataset\training_vol\vol01.mat
<class 'torch.Tensor'>


NameError: name 'np' is not defined